<a href="https://colab.research.google.com/github/sha-doow/ML/blob/main/Hate_Speech_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk.corpus import stopwords
import re
import string
from joblib import dump

# Load dataset
data = pd.read_csv("https://raw.githubusercontent.com/fenago/datasets/main/twitter.csv")

# Preprocessing
data["labels"] = data["class"].map({0: "Hate Speech", 1: "Offensive Language", 2: "No Hate and Offensive"})
data = data[["tweet", "labels"]]

nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword = set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)

# Vectorization
x = np.array(data["tweet"])
y = np.array(data["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Model Training
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Saving the model and vectorizer
dump(clf, 'hate_speech_model.pkl')
dump(cv, 'count_vectorizer.pkl')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['count_vectorizer.pkl']

In [3]:
# Ensure you have already loaded your model and vectorizer
from joblib import load
clf = load('hate_speech_model.pkl')
cv = load('count_vectorizer.pkl')

# Function to preprocess and predict
def predict_hate_speech(text):
    # Preprocess the text (ensure this is consistent with your training preprocessing)
    text = clean(text)  # Assuming 'clean' is your preprocessing function
    text_vectorized = cv.transform([text]).toarray()
    prediction = clf.predict(text_vectorized)
    return prediction[0]

# Test with some sample texts
sample_texts = ["I don't like you", "I hate the way you think"]
for text in sample_texts:
    prediction = predict_hate_speech(text)
    print(f"Tweet: {text}\nPrediction: {prediction}\n")

Tweet: I don't like you
Prediction: Offensive Language

Tweet: I hate the way you think
Prediction: No Hate and Offensive



In [1]:
pip install streamlit

In [4]:
import streamlit as st
from joblib import load

# Load the model and vectorizer
clf = load('hate_speech_model.pkl')
cv = load('count_vectorizer.pkl')

def hate_speech_detection(tweet):
    data = cv.transform([tweet]).toarray()
    prediction = clf.predict(data)
    return prediction[0]

st.title("Hate Speech Detection")
user_input = st.text_area("Enter a Tweet:")

if user_input:
    prediction = hate_speech_detection(user_input)
    st.write(f"Prediction: {prediction}")

2024-06-24 00:08:08.778 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-24 00:08:08.785 Session state does not function when running a script without `streamlit run`
